<a href="https://colab.research.google.com/github/michaelherediaperez/mecanica_de_solidos_un/blob/main/codigos/cap_02/02_08_02_ejemplos_vf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08-Ejemplo 1 (sección 2.8.2)

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Michael Heredia Pérez, <mherediap@unal.edu.co> | 2022-07-14 | Conversión de código de MAXIMA a Python|
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | 2022-09-13 | Mejorando los comentarios |
|Juan Nicolás Ramírez, <jnramirezg@unal.edu.co> | 2022-09-18 | Agregando cálculos de esf. y dir. pples |
|Juan Nicolás Ramírez, <jnramirezg@unal.edu.co> | 2022-09-19 | Adecuando y simplificando código |



Consideremos un punto $P$ de un sólido sometido los esfuerzos:

$\sigma_x=1$ Pa, 

$\sigma_y=3$ Pa,

$\sigma_z = \tau_{xy} = \tau_{xz} = 0$ Pa,

$\tau_{yz}= 2$ Pa.

Se pide:

* Plantear la matriz de tensiones $\underline{\underline{\sigma}}$ correspondiente.
* Calcular el polinomio característico asociado a $\underline{\underline{\sigma}}$.
* Calcular la dirección y magnitud de los esfuerzos principales.

 Importamos algunas funciones necesarias de ```numpy``` e importamos el submódulo ```linalg``` de ```numpy``` (álgebra lineal).

In [1]:
from numpy import array, poly, roots, sort, arctan, rad2deg, cross, transpose
import numpy.linalg as LA

## Matriz de tensiones
Definimos las componentes de esfuerzos dadas como variables numéricas:

In [2]:
sx  = 1
sy  = 3
sz  = 0
txy = 0
txz = 0
tyz = 2

Definimos la matriz de tensiones $\underline{\underline{\boldsymbol{\sigma}}}$ en tres dimensiones:

In [3]:
sigma = array([[sx,  txy, txz],
               [txy,  sy, tyz],
               [txz, tyz,  sz]])
sigma

array([[1, 0, 0],
       [0, 3, 2],
       [0, 2, 0]])

## Polinomio característico

Obtenemos los coeficientes del polinomio característico mediante ```numpy.poly()```:

In [4]:
poly(sigma)

array([ 1., -4., -1.,  4.])

Pero como el polinomio que definimos en el <font color='blue'>main.pdf</font> tiene la forma $-\sigma_n^3+I_1\sigma_n^2-I_2\sigma_n+I_3 = 0$, multiplicamos por -1:

In [5]:
-poly(sigma)

array([-1.,  4.,  1., -4.])

Lo interpretamos como:
$$-\sigma_n^3+ 4\sigma_n^2 + \sigma_n -4=0$$

Así, los invariantes de esfuerzos son $I_1 =4Pa$, $I_2=-1Pa^2$ y $I_3=-4Pa^3$.

Y mediante la función ```numpy.roots()``` calculamos la raíces del polinomio característico, es decir, los esfuerzos principales:

In [6]:
roots(poly(sigma))

array([ 4., -1.,  1.])

No olvidar que debemos garantizar por definición en el <font color='blue'>main.pdf</font> que:
$$\sigma_1 \geq \sigma_2 \geq \sigma_3$$

Por lo que a la hora de definir los esfuerzos principales en variables ```s1```, ```s2``` y ```s3``` debemos prestar atención.

## Esfuerzos y direcciones principales

Sin embargo, hay una forma directa de calcular los valores propios (esfuerzos principales) en ```numpy.linalg```, que llamamos ```LA```.

Para el cálculo de las direcciones, se requieren los vectores propios de la matriz $\underline{\underline{\sigma}}$. Usamos la función ```LA.eigh()``` que calcula tanto los valores ```valp``` como los vectores propios ```vecp```:

In [7]:
valp, vecp = LA.eigh(-sigma)  # Ponemos -sigma para que los valores propios queden ordenados de mayor a menor.

Los valores propios representan la magnitud de los esfuerzos principales $\sigma_1$ , $\sigma_2$ y $\sigma_3$:

In [8]:
valp

array([-4., -1.,  1.])

Almacenamos en memoria como ```s1```, ```s2``` y ```s3```, teniendo en cuenta que $\sigma_1 \geq \sigma_2 \geq \sigma_3$, o sea, $\sigma_1 = 4$Pa, $\sigma_2 = 1$Pa y $\sigma_3 = -1$Pa.

In [9]:
s1, s2, s3 = valp

In [10]:
vecp

array([[-0.        ,  1.        ,  0.        ],
       [-0.89442719,  0.        ,  0.4472136 ],
       [-0.4472136 ,  0.        , -0.89442719]])

Por lo que si transportenos la matriz ```vecp``` obtenemos en cada fila las direcciones principales.

In [11]:
ng1 = vecp.T[0]
ng2 = vecp.T[1]
ng3 = vecp.T[2]

In [12]:
LA.norm(ng1)

0.9999999999999999

In [13]:
LA.norm(ng2)

1.0

In [14]:
LA.norm(ng3)

0.9999999999999999

Los vectores propios representan las direcciones $\underline{\hat{n_1}}$ y $\underline{\hat{n_2}}$ de los esfuerzos principales:


In [15]:
cross(ng1, ng2)

array([ 0.        , -0.4472136 ,  0.89442719])

In [16]:
ng3

array([ 0.        ,  0.4472136 , -0.89442719])

## Verificación del sistema de coordenadas de la mano derecha

En MAXIMA se trabaja con la librería ```vect``` que define el producto cruz como la combinación del operador ```~``` y la función ```express()```; sin embargo en numpy basta con emplear la función ```numpy.cross```.

In [21]:
cross(ng1, ng2)

array([ 0.        , -0.4472136 ,  0.89442719])

In [20]:
ng3

array([ 0.        ,  0.4472136 , -0.89442719])

En particular, no se cumple con $\underline{\hat{n_1}}x\underline{\hat{n_2}}=\underline{\hat{n_3}}$, pero sí se cumple $\underline{\hat{n_1}}x\underline{\hat{n_2}}=-\underline{\hat{n_3}}$.

De esta manera, para tener una base que cumpla con la regla de la mano derecha, solo es necesario cambiarle el sentido a cualquiera de los tres vectores unitarios.

## Verificación de la no existencia de esfuerzos cortantes

Se construye la matriz de esfuerzos $\underline{\underline{\boldsymbol{\sigma}}}$ y de transformación $\boldsymbol{T}$:

In [18]:
T = vecp
T

array([[-0.        ,  1.        ,  0.        ],
       [-0.89442719,  0.        ,  0.4472136 ],
       [-0.4472136 ,  0.        , -0.89442719]])

La matriz de esfeurzos en el nuevo sistema de coordenadas, $\underline{\underline{\boldsymbol{\sigma}}}'$

In [28]:
sigmaP = transpose(T)@sigma@T
sigmaP.round(5)

array([[ 4.,  0.,  0.],
       [ 0.,  1.,  0.],
       [-0.,  0., -1.]])